## Initialize Project


In [1]:
from thematos import HyFI

if HyFI.is_colab():
    HyFI.mount_google_drive()

project_dir = HyFI.DotEnvConfig().DOTENV_DIR

h = HyFI.initialize(
    project_name="thematos",
    project_root=project_dir,
    logging_level="INFO",
    verbose=True,
)

print("project_dir:", h.project.root_dir)
print("project_workspace_dir:", h.project.workspace_dir)


/home/yjlee/.venvs/thematos/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:hyfi.utils.notebooks:Google Colab not detected.
INFO:hyfi.utils.notebooks:Extension autotime not found. Install it first.
INFO:hyfi.joblib.joblib:initialized batcher with <hyfi.joblib.batch.batcher.Batcher object at 0x7f070b0845b0>
INFO:hyfi.main.config:HyFi project [thematos] initialized


project_dir: /raid/cis/yjlee/workspace/projects/thematos
project_workspace_dir: /raid/cis/yjlee/workspace/projects/thematos/workspace


In [2]:
from thematos.datasets import Corpus

data_file = (
    h.project.root_dir / "workspace/datasets/processed/khmer_tokenized/train.parquet"
)
data_load = {"data_file": str(data_file)}
c = Corpus(data_load=data_load, id_col="id", text_col="text")

INFO:hyfi.task.batch:Initalized batch: corpus(2) in /raid/cis/yjlee/workspace/projects/thematos/workspace/topic/corpus


In [3]:
from thematos.models import WordPrior

data_file = h.project.root_dir / "workspace/datasets/word_prior.yaml"


w = WordPrior(data_file=str(data_file), verbose=True)
w.priors


INFO:hyfi.task.batch:Initalized batch: corpus(2) in /raid/cis/yjlee/workspace/projects/thematos/workspace/topic/corpus


{0: ['interest', 'rates', 'central', 'bank', 'nbc']}

In [10]:
from thematos.models import LdaModel

lda = LdaModel(corpus=c, wordprior=w, verbose=True)
lda.model_args.k = 10
lda.train_args.iterations = 50

INFO:hyfi.batch.batch:Setting seed to 4206844027
INFO:hyfi.batch.batch:Init batch - Batch name: model, Batch num: 0
INFO:hyfi.batch.batch:Init batch - Batch name: model, Batch num: 11
INFO:hyfi.task.batch:Initalized batch: model(11) in /raid/cis/yjlee/workspace/projects/thematos/workspace/topic/model


In [12]:
# lda.train()
lda.load(batch_num=10)


INFO:hyfi.task.batch:> Loading config for batch_name: model batch_num: 10
INFO:hyfi.task.batch:Loading config from /raid/cis/yjlee/workspace/projects/thematos/workspace/topic/model/configs/model(10)_config.yaml
INFO:hyfi.task.batch:Merging config with the loaded config
INFO:hyfi.task.batch:Updating config with config_kwargs: {}
INFO:hyfi.task.batch:Initalized batch: corpus(2) in /raid/cis/yjlee/workspace/projects/thematos/workspace/topic/corpus
INFO:hyfi.task.batch:Initalized batch: model(10) in /raid/cis/yjlee/workspace/projects/thematos/workspace/topic/model
INFO:hyfi.utils.iolibs:Processing [1] files from ['/raid/cis/yjlee/workspace/projects/thematos/workspace/topic/model/outputs/LDA_model(10)_k(10)-ll_per_word.csv']
INFO:hyfi.utils.datasets.load:Loading data from /raid/cis/yjlee/workspace/projects/thematos/workspace/topic/model/outputs/LDA_model(10)_k(10)-ll_per_word.csv
INFO:hyfi.utils.datasets.load: >> elapsed time to load data: 0:00:00.009628
INFO:hyfi.utils.iolibs:Processing [1

In [13]:
len(lda.topic_term_dists_df.columns)

61001

In [14]:
lda.doc_topic_dists_df.head()

,id,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9
0,501330943,0.031250,0.000128,0.064672,0.000136,0.037765,0.000073,0.001145,0.000134,0.014601,0.850096
1,501326169,0.215158,0.000866,0.001298,0.000922,0.000879,0.000493,0.001000,0.000911,0.001115,0.777357
2,501325554,0.025356,0.000128,0.078041,0.032611,0.008448,0.000073,0.017288,0.000134,0.045093,0.792828
3,501322478,0.000114,0.000119,0.043899,0.019885,0.000120,0.000068,0.000137,0.000125,0.027738,0.907795
4,501321914,0.024748,0.000106,0.713343,0.107948,0.066905,0.000061,0.000123,0.043344,0.002298,0.041124


In [16]:
topic_freq = dict(lda.model.get_topic_words(1, top_n=100))